In [1]:
import os
import random
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from io import BytesIO

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import CLIPProcessor, CLIPModel
from sentence_transformers import SentenceTransformer

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
CLIP_MODEL = "openai/clip-vit-base-patch32"
SENT_MODEL = "all-MiniLM-L6-v2"

In [3]:
sent_model = SentenceTransformer(SENT_MODEL, device=DEVICE)
clip_model = CLIPModel.from_pretrained(CLIP_MODEL).to(DEVICE)
clip_processor = CLIPProcessor.from_pretrained(CLIP_MODEL)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [4]:
import requests
def load_image_safe(url):
    try:
        resp = requests.get(url, timeout=8)
        img = Image.open(BytesIO(resp.content)).convert("RGB")
        return img
    except:
        return Image.new("RGB", (224, 224), color=(255,255,255))

In [5]:
def process_batch(batch_path, batch_id):
    df = pd.read_csv(batch_path)
    print(f"\nProcessing {batch_path} ... ({len(df)} samples)")

    # --- Text embeddings ---
    texts = df['catalog_content'].astype(str).tolist()
    text_embs = sent_model.encode(texts, batch_size=128, convert_to_numpy=True, show_progress_bar=True)
    np.save(f"embeddings/text_batch{batch_id}.npy", text_embs)

    # --- Image embeddings ---
    all_img_embs = []
    for i in tqdm(range(0, len(df), 32), desc=f"Batch {batch_id} CLIP"):
        imgs = [load_image_safe(u) for u in df['image_link'].iloc[i:i+32]]
        inputs = clip_processor(images=imgs, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            feats = clip_model.get_image_features(**inputs).cpu().numpy()
        # normalize
        feats = feats / (np.linalg.norm(feats, axis=1, keepdims=True) + 1e-8)
        all_img_embs.append(feats)
    img_embs = np.vstack(all_img_embs)
    np.save(f"embeddings/img_batch{batch_id}.npy", img_embs)
    print(f"Saved embeddings for batch {batch_id}")

In [6]:
!mkdir -p dataset

In [7]:
!mkdir -p embeddings

In [8]:
for i in range(1, 14):
    process_batch(f"dataset/test_batch_{i}.csv", i)


Processing dataset/test_batch_1.csv ... (6000 samples)


Batches:   0%|          | 0/47 [00:00<?, ?it/s]

Batch 1 CLIP: 100%|██████████| 188/188 [11:57<00:00,  3.82s/it]


Saved embeddings for batch 1

Processing dataset/test_batch_2.csv ... (6000 samples)


Batches:   0%|          | 0/47 [00:00<?, ?it/s]

...

Batch 3 CLIP: 100%|██████████| 188/188 [12:13<00:00,  3.90s/it]


Saved embeddings for batch 3

Processing dataset/test_batch_4.csv ... (6000 samples)


Batches:   0%|          | 0/47 [00:00<?, ?it/s]

Batch 4 CLIP: 100%|██████████| 188/188 [12:18<00:00,  3.93s/it]

Saved embeddings for batch 4


In [9]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
import joblib


In [10]:
!mkdir -p models

In [11]:
!mkdir -p final_embeddings

In [12]:
print("🔹 Loading trained model and scaler ...")
model = lgb.Booster(model_file="models/final_lightgbm_gpu.txt")
scaler = joblib.load("final_embeddings/numeric_scaler.pkl")

🔹 Loading trained model and scaler ...


In [17]:
num_batches = 13
all_test_df, all_text, all_img = [], [], []

for i in range(1, num_batches + 1):
    df_path = f"dataset/test_batch_{i}.csv"
    text_path = f"embeddings/text_batch{i}.npy"
    img_path  = f"embeddings/img_batch{i}.npy"

    if not os.path.exists(df_path):
        print(f"⚠️ Skipping {df_path} (not found)")
        continue

    print(f"Loading {df_path} ...")
    df = pd.read_csv(df_path)
    df["batch_id"] = i
    all_test_df.append(df)

    all_text.append(np.load(text_path))
    all_img.append(np.load(img_path))

test_df = pd.concat(all_test_df, ignore_index=True)
text_embs = np.vstack(all_text)
img_embs  = np.vstack(all_img)

print(f"✅ Combined test shape: {test_df.shape}, text_embs: {text_embs.shape}, img_embs: {img_embs.shape}")

Loading dataset/test_batch_1.csv ...
Loading dataset/test_batch_2.csv ...
Loading dataset/test_batch_3.csv ...
Loading dataset/test_batch_4.csv ...
Loading dataset/test_batch_5.csv ...
Loading dataset/test_batch_6.csv ...
Loading dataset/test_batch_7.csv ...
Loading dataset/test_batch_8.csv ...
Loading dataset/test_batch_9.csv ...
Loading dataset/test_batch_10.csv ...
Loading dataset/test_batch_11.csv ...
Loading dataset/test_batch_12.csv ...
Loading dataset/test_batch_13.csv ...
✅ Combined test shape: (75000, 5), text_embs: (75000, 384), img_embs: (75000, 512)


In [18]:
import re, unicodedata

def clean_text(x):
    if not isinstance(x, str): return ""
    x = unicodedata.normalize("NFKC", x).lower()
    x = re.sub(r"\s+", " ", x).strip()
    return x

test_df["catalog_content_clean"] = test_df["catalog_content"].apply(clean_text)
test_df["ipq"] = test_df["catalog_content_clean"].str.extract(r"(\d+)\s*(?:pack|pcs|pieces|count|ct|bottle|x)", expand=False)
test_df["ipq"] = pd.to_numeric(test_df["ipq"], errors="coerce").fillna(1)
test_df["title"] = test_df["catalog_content"].str.split(r"[:.]").str[0].fillna("")
test_df["len_title"] = test_df["title"].apply(lambda x: len(str(x)))
test_df["num_digits"] = test_df["catalog_content"].str.count(r"\d").fillna(0)
test_df["num_tokens"] = test_df["catalog_content_clean"].apply(lambda x: len(x.split()))
test_df["has_brand_keyword"] = test_df["catalog_content"].str.contains(
    r"brand|inc|co\.|limited|™|®", case=False, na=False
).astype(int)

num_cols = ["ipq", "len_title", "num_digits", "num_tokens", "has_brand_keyword"]
numeric_feats = test_df[num_cols].values.astype(np.float32)
num_scaled = scaler.transform(numeric_feats)

In [19]:
X_test = np.concatenate([text_embs, img_embs, num_scaled], axis=1)
print(f"🔹 Final test feature matrix: {X_test.shape}")

🔹 Final test feature matrix: (75000, 901)


In [20]:
print("🚀 Generating predictions on test set (GPU LightGBM)...")
pred_log = model.predict(X_test, num_iteration=model.best_iteration)
pred_price = np.expm1(pred_log)
pred_price = np.clip(pred_price, 0.01, None)  # ensure positiv

🚀 Generating predictions on test set (GPU LightGBM)...


In [21]:
submission = pd.DataFrame({
    "sample_id": test_df["sample_id"],
    "price": pred_price.round(2)
})

os.makedirs("submissions", exist_ok=True)
out_path = "submissions/test_out.csv"
submission.to_csv(out_path, index=False)
print(f"✅ Saved submission file: {out_path}")
print(submission.head())

✅ Saved submission file: submissions/test_out.csv
   sample_id  price
0      33127   4.93
1     198967  13.03
2     261251   2.02
3      55858  30.29
4     292686  65.76
